# turn nested namedtuples to dict for json storage

In [1]:
from collections import namedtuple


Spectrum = namedtuple(
    'Spectrum', 
    ['spectrum', 'abundance', 'ms_level', 'scan_number', 'precursor_mass', 'file_name'],
    defaults=[[], [], 0, -1, 0, '']
)


Kmer = namedtuple(
    'Kmer', 
    ['k', 'sequence', 'protein', 'start_position', 'end_position'], 
    defaults=[[0, '', '', -1, -1]]
)


BasicScoredKmer = namedtuple(
    'BasicScoredKmer', 
    ['b_score', 'y_score', 'kmer'],
    defaults=[0, 0, '']
)


ScoredKmer = namedtuple(
    'ScoredKmer', 
    ['b_score', 'y_score', 'kmer'],
    defaults=[0, 0, Kmer(0, '', '', -1, -1)]
)


AlignedScoredKmers = namedtuple(
    'AlignedScoreKmers',
    ['b_alignment', 'y_alignment', 'spectrum', 'protein', 'alignment_score', 'sequence', 'hybrid', 'hybrid_sequence'],
    defaults=[ScoredKmer(0, 0,  Kmer(0, '', '', -1, -1)), ScoredKmer(0, 0,  Kmer(0, '', '', -1, -1)), Spectrum([], [], 0, -1, 0, ''), '', 0, '', False, '']
)

In [4]:
print(type(AlignedScoredKmers))
print(type(namedtuple))
bsk = BasicScoredKmer(1, 2, 'abc')
print(type(bsk))

<class 'type'>
<class 'function'>
<class '__main__.BasicScoredKmer'>


In [15]:
def __jsonify_namedtuple(tup: namedtuple) -> dict:
        '''
        Recursive method to make any nested namedtuple json friendly

        Inputs:
            tup:    (namedtuple) thing to be flattened
        Outputs:
            (dict) json friendly namedtuple
        '''
        td = dict(tup._asdict())
        for key, value in td.items():
            if any([x in str(type(value)) for x in ['ScoredKmer', 'Kmer', 'AlignedScoredKmers', 'Specturm']]):
                td[key] = __jsonify_namedtuple(value)
        return td

In [20]:
spec = Spectrum([1, 2, 3], [0.1, 0.2, 0.3], 2, 1, 3, 'file.mzML')
kmer1 = Kmer(3, 'abc', 'prot1', 2, 4)
kmer2 = Kmer(5, 'defgh', 'prot2', 5, 9)
sk1 = ScoredKmer(1, 1, kmer1)
sk2 = ScoredKmer(2, 1.3, kmer2)
ask = AlignedScoredKmers(sk1, sk2, spec, 'prot1-prot2-hybrid', 4, 'abcdefgh', True, 'abc-defgh')

NameError: name 'AlignedScoreKmers' is not defined

In [12]:
print(__jsonify_namedtuple(spec))

{'spectrum': [1, 2, 3], 'abundance': [0.1, 0.2, 0.3], 'ms_level': 2, 'scan_number': 1, 'precursor_mass': 3, 'file_name': 'file.mzML'}


In [13]:
print(__jsonify_namedtuple(kmer1))

{'k': 3, 'sequence': 'abc', 'protein': 'prot1', 'start_position': 2, 'end_position': 4}


In [16]:
print(__jsonify_namedtuple(sk1))

{'b_score': 1, 'y_score': 1, 'kmer': {'k': 3, 'sequence': 'abc', 'protein': 'prot1', 'start_position': 2, 'end_position': 4}}


In [19]:
print(__jsonify_namedtuple(ask))

AttributeError: 'tuple' object has no attribute '_asdict'